# Assignment 2 Part A - Convolutional Neural Networks

Welcome to the second assignment!

- This assignment is shorter
    - There are only two questions (implementing `Conv1d`)
    - We did this to give you more time for training in part B. Big computer vision models!

Before we jump into the code, let's try to understand convolutions.

# Section 1: Introduction to 1D Convolutional Layers
Of course, we already cover convolutions in lecture, but for your convenience/understanding, this notebook will provide an alternate explanation with examples directly relevant to the questions.

Feel free to skip to the coding if you already understand convolutions well. We provide pseudocode there if you need it.

## 1.1: Problem Motivation

- Convolutions are great for problems that have **close-range relationships** between positions in your input.
- Most popular application of convolutions is 2D convolutions for image processing tasks
    - For images, the close-range relationships are between pixel positions (x coordinate and y coordinate). In other words, pixels that are near each other tend to be related in some significant way.
- But convolutions are also used for 1-dimensional applications
    - Examples: Audio-processing or stock price prediction

In this assignment, you'll be implementing a 1D convolutional layer.

## 1.2: A Conceptual Intro to 1D Convolutions

So what are convolutions and how do they capture close-range relationships?

The most intuitive visualization of them is by visualizing a sliding window.

<div style="text-align:center">
    <img src="images/spectrogram.png" width="500"/>
</div>

Imagine you're given this input; a spectrogram just like the ones in part B of the first assignment. It's shaped `(num_frames, num_channels)`.

<div style="text-align:center">
    <img src="images/spectrogram_animation.gif" width="500"/>
</div>

**Explanation**:
- We move the weight matrix ('kernel') along the input, and performing a `tensordot` (inner product) each time we move it
- We move a fixed distance every time.
    - This distance is called our **stride**
- We then concatenate the outputs together to get a final output.

**Main Idea**:

By using a sliding window, a convolutional layer **captures the short-distance relationships between each snippet of the input that the kernel sees**. The weights will ideally end up learning what to bring out in these short-distance relationships for downstream layers to process.

## 1.3: A Detailed Intro to Convolutions

Now for a concrete example.

Let's begin by introducing the input data and our parameters.

### 1.3.1: Input Data:

<div style="text-align:center">
    <img src="images/input.png" width="400"/>
</div>

Our input is a batch of two 2D matrices. You can imagine each horizontal slice (member of the batch) being a single spectrogram. Assume for now that every spectrogram is the same size so we can easily stack them into a batch.

You can also interpret `input_size` as the time dimension, which will be true in many applications of `Conv1d` but not all.

---

### 1.3.2: Convolutional Layer

<div style="text-align:center">
    <img src="images/conv1d.png" width="400"/>
</div>

Our `Conv1d` layer has a weight matrix and bias vector, just like a `Linear` layer. The weight matrix is our kernel. For convolutions, we use the terms "weights" or "kernels" interchangably.

**Notice the params we initialized the layer with:**

- `in_channels` is predetermined (based on how many channels our given input has)
- We choose the rest:
    - We choose `out_channels` based on intuition about how much complexity from the input's `in_channels` we want to isolate/preserve.
    - We choose `kernel_size` based on how close-range the relationships are.
        - The larger the kernel, the longer-range relationships you can encode (with caveats).
    - We choose `stride` based on how much overlap we want between outputs.
        - If `stride < kernel_size`, we'll end up processing some parts of the input multiple times (which is often a good thing).

---

### 1.3.3: The Algorithm

Now that we've covered the input and parameters, let's now cover how they're used.

<div style="text-align:center">
    <img src="images/step_1.png" width="400"/>
</div>

Just like our simple example above, we start at the beginning of the input and take a small slice to multiply against our kernel.

<div style="text-align:center">
    <img src="images/conv_details.png" width="800"/>
</div>

We [`tensordot`](https://numpy.org/doc/stable/reference/generated/numpy.tensordot.html) (similar to a `matmul`, see appendix for some more details) along the **last two axes of our kernel and input slice** (`(in_channels, kernel_size)` for both). We then add the bias vector to the result, and then store it, just like in our simple example.

We then `stride=2` units over and repeat this process to get our final output.

<div style="text-align:center">
    <img src="images/conv_animation.gif" width="800"/>
</div>

Notice that we only stride until the kernel no longer fits onto the input.

In other words, **because of how we chose our parameters, `Conv1d` ignores the very last frame of this input**.

<div style="text-align:center">
    <img src="images/last_step.png" width="450"/>
</div>

This only ever happens at the edges of your input (borders of an image, last few frames of an audio clip). We try to avoid this by either tuning our parameters or by **padding** zeroes to the edges of the input (which we won't get into for now).

<div style="text-align:center">
    <img src="images/processed_twice.png" width="450"/>
</div>

Also, note that these two slices at index 2 and 5 were seen by the kernel twice. Again, this is usually not a bad thing, and is often done intentionally. 

---

### 1.3.4: Calculating Output Size
Last note: when moving the kernel along, you *could* implement this by iterating until the kernel no longer fits (probably using a `while` loop until some `break` condition), but `while` loops can be hard to debug and aren't easily parallelizable.

Instead, you can actually **calculate how many output slices you'll generate and just iterate for exactly that many slices**.

Here's the formula to figure out how many output slices you'll end up with:

$$\begin{align*}
    \text{output\_size} = \left \lfloor{\frac{\text{input\_size} - \text{kernel\_size}}{\text{stride}}}\right \rfloor + 1
\end{align*}$$

$$\begin{align*}
    &\text{Where $\left \lfloor{x}\right \rfloor$ is the floor function (i.e. round down) applied to some float $x$}
\end{align*}$$

So in this case, our `output_size=4`.

## 1.4: Summary

Given an input, for each slice defined by the stride a `Conv1d` layer takes the inner products between the weights and the slice, and then adds the biases.

- The size of each slice is determined by the `kernel_size`
- The location of the beginning of the next slice is determined by the `stride`
- The number of slices is determined by the `input_size`, the `kernel_size`, and `stride`, in the formula above.

That's it! To save you time, we'll give you pseudocode for tackling this below, although ideally you should try to implement this based on your own understanding of the intro.

**Make sure to run the cell below to import things!**

In [1]:
# Make sure to run this cell without modifying anything in it!
import numpy as np

# Import the code in `mytorch/nn.py`
from mytorch import nn

# These iPython functions make it so imported code is automatically reimported here if they are changed
%load_ext autoreload
%autoreload 2

## Question 1: `Conv1d.forward()`

Finally, we can begin coding. Make sure to load in the imports in the previous cell.

**First**, in `mytorch/nn.py`, complete `get_conv1d_output_size()` using the formula in Section 1.3.4 of this notebook. Should just be one line of code.

**Second**, in `mytorch/nn.py`, complete `Conv1d.forward()` using the pseudocode below.


**Pseudocode**

- Pre-calculate and store the number of output slices
- Pre-declare an output array filled with zeros, shaped `(batch_size, out_channels, output_size)`
    - This is where we'll store the results of each `tensordot`
- For each output slice to calculate:
    - Determine the beginning/end index of the current input slice
        - Hint: `beg_index = i * stride`, what is `end_index`?
    - Do a `tensordot` between the weight matrix and the 2nd/3rd axes of the input
    - Add the bias to the result
    - Store the result in the appropriate slice of the output array (hint: index along the last axis of `out`)
- Return the output array

Done. Hopefully it's clear how the above pseudocode accomplishes everything we discuss in our long intro.

In [2]:
def test_conv1d_forward_1(Conv1d):
    # Same params as in the introduction
    in_channels = 2
    out_channels = 3
    kernel_size = 2
    stride = 1

    # Declare layer, weights, and biases (values initialized to increasing integers for consistency)
    layer = Conv1d(in_channels, out_channels, kernel_size, stride)
    layer.weight = np.array([[[0, 1],
                              [2, 3]],

                             [[4, 2],
                              [-2, -4]],

                             [[3, -4],
                              [5, 2]]])

    layer.bias = np.array([0, 1, 2])

    # Declare example input (increasing integers)
    batch_size = 2
    input_size = 4

    #batch, in_ch, input
    x = np.array([[[0, 1, -3, -1],
                   [-2, 3, -2, 1]], 

                  [[4, 5, -2, 3],
                   [2, -1, -4, 7]]])

    out = layer.forward(x)
    return out


test_conv1d_forward_1(nn.Conv1d)

array([[[  6.,  -3.,  -2.],
        [ -5.,   1., -13.],
        [ -6.,  28., -11.]],

       [[  6., -16.,  16.],
        [ 27.,  35., -21.],
        [  2.,  12., -22.]]])

In [3]:
from tests import test_conv1d_forward_1, test_conv1d_forward_2, test_conv1d_forward_3

answer_1 = test_conv1d_forward_1(nn.Conv1d)
answer_2 = test_conv1d_forward_2(nn.Conv1d)
answer_3 = test_conv1d_forward_3(nn.Conv1d)

In [4]:
answer_1

array([[[  6.,  -3.,  -2.],
        [ -5.,   1., -13.],
        [ -6.,  28., -11.]],

       [[  6., -16.,  16.],
        [ 27.,  35., -21.],
        [  2.,  12., -22.]]])

In [5]:
answer_2

array([[[  4324.],
        [ 10949.],
        [ 17574.],
        [ 24199.],
        [ 30824.]],

       [[ 10948.],
        [ 31397.],
        [ 51846.],
        [ 72295.],
        [ 92744.]],

       [[ 17572.],
        [ 51845.],
        [ 86118.],
        [120391.],
        [154664.]]])

In [6]:
answer_3

array([[[ -225. ,  -252.5,  -280. ,  -307.5,  -335. ],
        [   26. ,    23.5,    21. ,    18.5,    16. ],
        [  277. ,   299.5,   322. ,   344.5,   367. ]],

       [[ -912.5,  -940. ,  -967.5,  -995. , -1022.5],
        [  -36.5,   -39. ,   -41.5,   -44. ,   -46.5],
        [  839.5,   862. ,   884.5,   907. ,   929.5]],

       [[-1600. , -1627.5, -1655. , -1682.5, -1710. ],
        [  -99. ,  -101.5,  -104. ,  -106.5,  -109. ],
        [ 1402. ,  1424.5,  1447. ,  1469.5,  1492. ]],

       [[-2287.5, -2315. , -2342.5, -2370. , -2397.5],
        [ -161.5,  -164. ,  -166.5,  -169. ,  -171.5],
        [ 1964.5,  1987. ,  2009.5,  2032. ,  2054.5]]])

In [7]:
# TODO: Run this cell to evaluate your code on all three unit tests.

If you passed the tests above, assign the string "Question 1 passed" to asn1 in the code cell below.

In [8]:
### GRADED

ans1 = None

# YOUR CODE HERE
# raise NotImplementedError()
ans1 = "Question 1 passed"
asn1 = "Question 1 passed"

# Section 2: `Conv1d.backward()`

## 2.1: Problem Motivation

Backprop for `Conv1d` is a little harder than it was for `Linear`.

Remember that the purpose of backprop is to determine how each parameter (weights and biases) affected the loss. We also calculate the gradient w.r.t. the input, in order to pass it along to earlier layers in the network.

The challenge comes from the fact that our weight tensor was used multiple times - each time it slid along the input and did a `tensordot`, its params affected the output at that part. So we need to calculate and sum up the influence of each param every time it was used.

**In summary**, just like `Linear.backward()`, the goal of `Conv1d.backward()` is to figure out how the params affected the loss and also pass the gradient w.r.t. the input backwards. This gets complicated because our params were used multiple times.

## 2.2: A Note

Instead of creating a detailed explanation in this notebook like we did for `Conv1d.forward()`, we've decided to just provide pseudocode.

We do this because:
1. An extensive conceptual explanation is already given in lecture
2. The explanation is pretty technical and explaining it again here doesn't really add much
    - The main takeaway is what we describe in the summary above

## Question 2: `Conv1d.backward()`


In `mytorch/nn.py`, complete `Conv1d.backward()`.

This will involve:

1. Iteratively calculating and `return`ing `dx`
2. Iteratively calculating and storing `grad_weight`
3. Calculating and storing `grad_bias`

Pseudocode:
- Make array filled with 0's, the same shape as the original input `x`
- For each slice in the number of output slices:
    - Calculate the beginning/end index of our current slice, exactly like we did in `forward()`
    - Add to `dx[:,:,b:e]` using the `+=` operator:  
        - `tensordot` between `delta[:,:,i]` and `self.weight` along `axes=(1)`
        - In other words, we're accumulating the influence of slice `i` of the output on indices `[b:e]` of the input. We use a `+=` because some slices of the input may have been used multiple times, so we just sum up their influences
    - Add to `self.grad_weight` using the `+=` operator:
        - `tensordot` between `delta[:,:,i].T` and `self.x[:,:,b:e]` along `axes=(1)`
        - We add to the entire weight's gradient because the entire kernel was used for this slice
        - Again, it's `+=` because the kernel possibly saw parts of the input multiple times, so we just sum up its total influence on those parts
- We can calculate the bias's gradient in one line of code:
    - Set `self.grad_bias` equal to the sum of `delta` along axes `(0,2)`
    - This works because the bias affected each part of the output the same way, so we just need its total influence.

Done!

In [9]:
def test_conv1d_backward_1(Conv1d):
    # Same params as in the introduction
    in_channels = 2
    out_channels = 3
    kernel_size = 2
    stride = 1

    # Declare layer, weights, and biases (values initialized to increasing integers for consistency)
    layer = Conv1d(in_channels, out_channels, kernel_size, stride)
    layer.weight = np.array([[[0, 1],
                              [2, 3]],

                             [[4, 2],
                              [-2, -4]],

                             [[3, -4],
                              [5, 2]]])
    layer.bias = np.array([0, 1, 2])

    # Declare example input
    batch_size = 2
    input_size = 4

    # Run the forward pass
    x = np.array([[[0, 1, -3, -1],
                   [-2, 3, -2, 1]], 

                  [[4, 5, -2, 3],
                   [2, -1, -4, 7]]])
    layer.forward(x)

    # Make up a gradient of loss w.r.t. output of this layer 
    delta = np.array([[[2, -1, -1],
                       [2, -2, 3],
                       [2, 2, 1]],

                      [[-0, 0, -3],
                       [2, 7, -1],
                       [8, 7, 2]]])

    grad_x = layer.backward(delta)

    return grad_x, layer.grad_weight, layer.grad_bias

test_conv1d_backward_1(nn.Conv1d)

(array([[[ 14.,  -4.,   2.,   1.],
         [ 10.,  14.,   6., -13.]],
 
        [[ 32.,  21., -12., -13.],
         [ 36.,  29.,  -8.,  -1.]]]), array([[[  8.,  -3.],
         [  7., -14.]],
 
        [[ 34.,  -2.],
         [-15., -24.]],
 
        [[ 62.,  27.],
         [  1., -19.]]]), array([-3, 11, 22]))

In [10]:
from tests import test_conv1d_backward_1, test_conv1d_backward_2, test_conv1d_backward_3

answer_1 = test_conv1d_backward_1(nn.Conv1d)
answer_2 = test_conv1d_backward_2(nn.Conv1d)
answer_3 = test_conv1d_backward_3(nn.Conv1d)

In [11]:
answer_1

(array([[[ 14.,  -4.,   2.,   1.],
         [ 10.,  14.,   6., -13.]],
 
        [[ 32.,  21., -12., -13.],
         [ 36.,  29.,  -8.,  -1.]]]), array([[[  8.,  -3.],
         [  7., -14.]],
 
        [[ 34.,  -2.],
         [-15., -24.]],
 
        [[ 62.,  27.],
         [  1., -19.]]]), array([-3, 11, 22]))

In [12]:
answer_2

(array([[[ 720.,  730.,  740.,  750.,  760.,  770.,  780.,  790.],
         [ 800.,  810.,  820.,  830.,  840.,  850.,  860.,  870.],
         [ 880.,  890.,  900.,  910.,  920.,  930.,  940.,  950.]],
 
        [[1920., 1955., 1990., 2025., 2060., 2095., 2130., 2165.],
         [2200., 2235., 2270., 2305., 2340., 2375., 2410., 2445.],
         [2480., 2515., 2550., 2585., 2620., 2655., 2690., 2725.]],
 
        [[3120., 3180., 3240., 3300., 3360., 3420., 3480., 3540.],
         [3600., 3660., 3720., 3780., 3840., 3900., 3960., 4020.],
         [4080., 4140., 4200., 4260., 4320., 4380., 4440., 4500.]]]),
 array([[[ 600.,  615.,  630.,  645.,  660.,  675.,  690.,  705.],
         [ 720.,  735.,  750.,  765.,  780.,  795.,  810.,  825.],
         [ 840.,  855.,  870.,  885.,  900.,  915.,  930.,  945.]],
 
        [[ 672.,  690.,  708.,  726.,  744.,  762.,  780.,  798.],
         [ 816.,  834.,  852.,  870.,  888.,  906.,  924.,  942.],
         [ 960.,  978.,  996., 1014., 1032., 1050.

In [13]:
answer_3

(array([[[  12.5,    5. ,   -2.5,  -10. ,  -17.5],
         [  27.5,   23. ,   18.5,   14. ,    9.5],
         [  42.5,   41. ,   39.5,   38. ,   36.5],
         [  57.5,   59. ,   60.5,   62. ,   63.5],
         [  72.5,   77. ,   81.5,   86. ,   90.5]],
 
        [[-100. , -107.5, -115. , -122.5, -130. ],
         [ -40. ,  -44.5,  -49. ,  -53.5,  -58. ],
         [  20. ,   18.5,   17. ,   15.5,   14. ],
         [  80. ,   81.5,   83. ,   84.5,   86. ],
         [ 140. ,  144.5,  149. ,  153.5,  158. ]],
 
        [[-212.5, -220. , -227.5, -235. , -242.5],
         [-107.5, -112. , -116.5, -121. , -125.5],
         [  -2.5,   -4. ,   -5.5,   -7. ,   -8.5],
         [ 102.5,  104. ,  105.5,  107. ,  108.5],
         [ 207.5,  212. ,  216.5,  221. ,  225.5]],
 
        [[-325. , -332.5, -340. , -347.5, -355. ],
         [-175. , -179.5, -184. , -188.5, -193. ],
         [ -25. ,  -26.5,  -28. ,  -29.5,  -31. ],
         [ 125. ,  126.5,  128. ,  129.5,  131. ],
         [ 275. ,  279

If you passed the tests above, assign the string "Question 2 passed" to asn2 in the code cell below.

In [15]:
### GRADED

ans2 = None

# YOUR CODE HERE
# raise NotImplementedError()
ans3 = 'Question 2 passed'
asn3 = 'Question 2 passed'

# Appendix

## `tensordot`

[NumPy documentation](https://numpy.org/doc/stable/reference/generated/numpy.tensordot.html)

Similar to dot products/matrix multiplication. 

Given two tensors $A$ and $B$ and an axis/axes you want to multiply across, we perform an element-wise multiplication along the desired axis/axes of the two tensors, then sum each of the products.

In a sense, you 'eliminate' the axes you specify. So if you tensordot along the 2nd & 3rd axes of tensors shaped `(2, 4, 3)` and `(5, 4, 3)`, your output will be shaped `(2, 5)`; the first axes 'remaining' after eliminating the second and third axes of both tensors.